In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))

In [4]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(lambda x: (x[:-1], x[1:]))

In [6]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [7]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [8]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [10]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [15]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([54, 61, 58, 35, 26, 33, 24, 24, 54,  6, 11, 42,  9, 43, 46, 24, 42,
       14, 44, 48, 20, 50,  3, 48, 30, 45, 59, 36,  3, 41, 45, 22,  0, 16,
       43, 13, 17, 55, 23, 24, 10, 31, 63, 28,  5, 25, 45,  1, 52, 57, 30,
       54, 59, 56,  2,  0,  7,  1,  4, 13, 47, 39, 15, 11, 28, 39, 25, 57,
       58, 57,  5, 10, 48, 54, 64, 15,  3, 41, 43, 15, 52, 34,  0, 45, 46,
       43, 63,  6, 36, 26, 30,  0, 28, 50, 32, 42, 40,  1, 64, 26])

In [11]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1755247


In [12]:
model.compile(optimizer='adam', loss=loss)

In [13]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [14]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 10s 60ms/step - loss: 2.6654
Epoch 2/10
172/172 [==============================] - 10s 60ms/step - loss: 1.9618
Epoch 3/10
172/172 [==============================] - 10s 60ms/step - loss: 1.6942
Epoch 4/10
172/172 [==============================] - 10s 61ms/step - loss: 1.5460
Epoch 5/10
172/172 [==============================] - 10s 61ms/step - loss: 1.4587
Epoch 6/10
172/172 [==============================] - 10s 61ms/step - loss: 1.3985
Epoch 7/10
172/172 [==============================] - 11s 61ms/step - loss: 1.3523
Epoch 8/10
172/172 [==============================] - 11s 61ms/step - loss: 1.3126
Epoch 9/10
172/172 [==============================] - 11s 61ms/step - loss: 1.2780
Epoch 10/10
172/172 [==============================] - 11s 62ms/step - loss: 1.2454


In [15]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [16]:
def generate_text(model, start_string):
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [17]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I do do.

HORTENSIO:
O to know you to the prone, peace, and gentle Vincentio;' have I well, France,
Is this devil annot and word of love can care;
For I will appear
To fill your successur with one in this right.

KING RICHARD III:
Meny hand! I speak out think,
So finger and as tworny,
That may sting a sorry of his.

G give left thereof will make it a king.

KING RICHARD III:
And that we wear and after her face she is,
Some foil of their eye that pleasant in the time that he shuns;
Or, my poor wither, I will, my lord, and straight
To be stingingly let him lend yet disgrace.
Be come nither in the man. VONERIO:
My lords, nurse again, or we know, that is myself
To flatter with greedonest leasts,
That after dunes. We came here to your sisses at the cheeks
you wot the coa life coasting in his life in hand,
Hangs will seem'd 'gainst an ond of the trades of the earth
shall but service after't,
That, forbid you to the dexiances
Her hearting hich take a secret placity of his line
Be thusb